In [ ]:
!pip install getdaft --pre --extra-index-url https://pypi.anaconda.org/daft-nightly/simple
!pip install Pillow torch torchvision

```{hint}
✨✨✨ **Run this notebook on Google Colab** ✨✨✨

You can [run this notebook yourself with Google Colab](https://colab.research.google.com/github/Eventual-Inc/Daft/blob/main/tutorials/mnist.ipynb)!
```

# MNIST Daft Tutorial

The MNIST Dataset is a "large database of handwritten digits that is commonly used for training various image processing systems".

## Loading Data

This is a JSON file containing all the data for the MNIST test set. Let's load it up into a DaFt Dataframe!

In [1]:
from daft import DataFrame, col, udf

URL = "https://github.com/Eventual-Inc/mnist-json/raw/master/mnist_handwritten_test.json.gz"
images_df = DataFrame.read_json(URL)

2023-02-01 10:55:27.447 | INFO     | daft.context:runner:85 - Using PyRunner


To peek at the dataset, simply have your notebook display the images_df that was just created.

In [2]:
images_df

imagePY[list],labelINTEGER


In [3]:
images_df.show(10)

imagePY[list],labelINTEGER
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",7
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",4
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",4
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",9
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",5
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",9


You just loaded your first DaFt Dataframe! It consists of two columns:
1. The "image" column is a Python column of type `list` - where it looks like each row contains a list of digits representing the pixels of each image
2. The "label" column is an Integer column, consisting of just the label of that image.

## Processing Columns with User-Defined Functions (UDF)

It seems our JSON file has provided us with a one-dimensional array of pixels instead of two-dimensional images. We can easily modify data in this column by instructing DaFt to run a method on every row in the column like so:

In [4]:
import numpy as np

images_df = images_df.with_column(
    "image_2d",
    col("image").apply(lambda l: np.array(l).reshape(28, 28), return_type=np.ndarray),
)

In [5]:
images_df.show(10)

imagePY[list],labelINTEGER,image_2dPY[ndarray]
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",7,"<np.ndarrayshape=(28, 28)dtype=int64>"
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2,"<np.ndarrayshape=(28, 28)dtype=int64>"
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,"<np.ndarrayshape=(28, 28)dtype=int64>"
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0,"<np.ndarrayshape=(28, 28)dtype=int64>"
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",4,"<np.ndarrayshape=(28, 28)dtype=int64>"
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,"<np.ndarrayshape=(28, 28)dtype=int64>"
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",4,"<np.ndarrayshape=(28, 28)dtype=int64>"
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",9,"<np.ndarrayshape=(28, 28)dtype=int64>"
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",5,"<np.ndarrayshape=(28, 28)dtype=int64>"
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",9,"<np.ndarrayshape=(28, 28)dtype=int64>"


Great, but we can do one better - let's convert these two-dimensional arrays into Images. Computers speak in pixels and arrays, but humans do much better with visual patterns!

To do this, we can leverage the `.apply` expression method. Similar to the `.as_py` method, this allows us to run a single function on all rows of a given column, but provides us with more flexibility as it takes as input any arbitrary function.

In [6]:
from PIL import Image

images_df = images_df.with_column("pil_image", col("image_2d").apply(lambda arr: Image.fromarray(arr.astype(np.uint8)), return_type=Image.Image))

In [7]:
images_df.show(10)

imagePY[list],labelINTEGER,image_2dPY[ndarray],pil_imagePY[Image]
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",7,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APA4Lea6nSC3hkmmc4SONSzMfQAdadc2dzZSmK7tpoJB/BKhU/kahoq1pupXuj6hDf6dcyW13CSY5YzhlyCDj8CRXXWvxe8b20SxtrH2lVOQbqCOU9OmWUn/APVXUfEfxBqCfDzSNJ16S2uNd1JxqEqpbohtIMYjQbQBlsEnv1HpXj9Fdx8OvDNlqNxe+IdeVh4e0VPPucLnznyNkQ/3j1/LjOa57xPr9z4n8R3usXQ2vcSZVB0jQcKo9gABWRRXSxeOdXt/A0nhGAW0WnSzGaZ1j/ey8g7SxOMZA6AHjrXNUV//2Q=="" alt="""" />"
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APAY42lkWNBl2IVR6k17BB8L/BemalZaB4h8TXr+I7oKhtNNi3rDI3TcdpyB36HvwK8z8UaL/wAI74o1LRxOs4s52iEg/iAPGff196yaK9d+H1lbeCPCdz8SNZjWW5bdBpFtJw0jn5TIDn/eHToCe4ryvUb+51XUrnULyQyXNzK0srnuzHJqtRX0J4utvBHxCXSLez+INlo+m2dqqQ2EkQCIf7xLMoB24XB9Pc1wp8D+AdMc/wBrfEaCbazDy9Os2l3YHGHBIHPtj3riNfg0a31V00G8ubuw2qUkuYhG+ccgge9ZlFFFf//Z"" alt="""" />"
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APn+lALEAAkngAd62PFGgjw1rR0trrz5o4Ynn/d7PLkZAzR9TnbnGfboKxqK634a6PHrPjrT1uAhs7Mm9ut/TyovmOfrgD8awte1M614h1LVCGX7ZcyThWOSoZiQM+2cVn0V6J4RFvo/wu8X63NKi3F6qaVarxuYsQ0nvjaR+XevO6KKKKK//9k="" alt="""" />"
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APn+u6+Hnwx1Lx9cTlZmsLGGMn7W8JdWfOAijIz3yc8Y+lcdqFjPpt9NaXMbJJE5U7lIzg4yM9qrVJbwtcXMUKqzNI4QBRkkk44r6F8c/Em3+GdnD4I8KWyi5sYVWS4lXiEsA/TGHZg24nplu5zjC11T8SPgyfFupp5WtaK7RG4Cqi3SFl4J46AjA9QcferxOtDQr9NK8Q6ZqMiF0tLuKdlHUhXDEfpX0VqHgHwJ461y88a3niUy2Ny0P7uKdIY0KoqbXZgTzt6fKetcF8VfiLpOo6Ta+E/CStBpNsx+0MkflpKRjaqgdVzkkkcnB9z5FRRRRX//2Q=="" alt="""" />"
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",4,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAx

Amazing! This looks great and we can finally get some idea of what the dataset truly looks like.

## Running a model with Stateful UDFs

Next, let's try to run a deep learning model to classify each image. Models are expensive to initialize and load, so we want to do this as few times as possible, and share a model across multiple invocations. Here's how we can do it with "Stateful UDFs".

For the convenience of this quickstart tutorial, we pre-trained a model using a PyTorch-provided example script and saved the trained weights at https://github.com/Eventual-Inc/mnist-json/raw/master/mnist_cnn.pt.  We need to define the same deep learning model "scaffold" as the trained model that we want to load (this part is all PyTorch and is not specific at all to DaFt)

In [8]:
###
# Model was trained using a script provided in PyTorch Examples: https://github.com/pytorch/examples/blob/main/mnist/main.py
###

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.hub
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

Now comes the fun part - we can define a Stateful UDF using a class and the `@udf` decorator. This lets us use the class' `__init__` method to perform any expensive initializations such as downloading the model weights and loading the model.

In [9]:
@udf(return_type=int)
class ClassifyImages:

    def __init__(self):
        # Perform expensive initializations - create the model, download model weights and load up the model with weights
        self._model = Net()
        state_dict = torch.hub.load_state_dict_from_url("https://github.com/Eventual-Inc/mnist-json/raw/master/mnist_cnn.pt")
        self._model.load_state_dict(state_dict)

    def __call__(self, images_2d_col: np.ndarray):
        normalized_image_2d = images_2d_col / 255
        normalized_image_2d = normalized_image_2d[:, np.newaxis, :, :]
        classifications = self._model(torch.from_numpy(normalized_image_2d).float())
        return classifications.detach().numpy().argmax(axis=1)

/Users/jaychia/code/Daft/daft/udf.py:88: UserWarning: DEPRECATION WARNING: @udf will be deprecated in 0.1.0 in favor of @polars_udf which is much more efficient and handles Null/NaN semantics correctly
  warnings.warn(


Using this Stateful Class UDF is really easy, we simply run it on the columns that we want to process:

In [10]:
classified_images_df = images_df.with_column("model_classification", ClassifyImages(col("image_2d")))

classified_images_df.show(10)

imagePY[list],labelINTEGER,image_2dPY[ndarray],pil_imagePY[Image],model_classificationINTEGER
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",7,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APA4Lea6nSC3hkmmc4SONSzMfQAdadc2dzZSmK7tpoJB/BKhU/kahoq1pupXuj6hDf6dcyW13CSY5YzhlyCDj8CRXXWvxe8b20SxtrH2lVOQbqCOU9OmWUn/APVXUfEfxBqCfDzSNJ16S2uNd1JxqEqpbohtIMYjQbQBlsEnv1HpXj9Fdx8OvDNlqNxe+IdeVh4e0VPPucLnznyNkQ/3j1/LjOa57xPr9z4n8R3usXQ2vcSZVB0jQcKo9gABWRRXSxeOdXt/A0nhGAW0WnSzGaZ1j/ey8g7SxOMZA6AHjrXNUV//2Q=="" alt="""" />",7
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APAY42lkWNBl2IVR6k17BB8L/BemalZaB4h8TXr+I7oKhtNNi3rDI3TcdpyB36HvwK8z8UaL/wAI74o1LRxOs4s52iEg/iAPGff196yaK9d+H1lbeCPCdz8SNZjWW5bdBpFtJw0jn5TIDn/eHToCe4ryvUb+51XUrnULyQyXNzK0srnuzHJqtRX0J4utvBHxCXSLez+INlo+m2dqqQ2EkQCIf7xLMoB24XB9Pc1wp8D+AdMc/wBrfEaCbazDy9Os2l3YHGHBIHPtj3riNfg0a31V00G8ubuw2qUkuYhG+ccgge9ZlFFFf//Z"" alt="""" />",2
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APn+lALEAAkngAd62PFGgjw1rR0trrz5o4Ynn/d7PLkZAzR9TnbnGfboKxqK634a6PHrPjrT1uAhs7Mm9ut/TyovmOfrgD8awte1M614h1LVCGX7ZcyThWOSoZiQM+2cVn0V6J4RFvo/wu8X63NKi3F6qaVarxuYsQ0nvjaR+XevO6KKKKK//9k="" alt="""" />",1
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APn+u6+Hnwx1Lx9cTlZmsLGGMn7W8JdWfOAijIz3yc8Y+lcdqFjPpt9NaXMbJJE5U7lIzg4yM9qrVJbwtcXMUKqzNI4QBRkkk44r6F8c/Em3+GdnD4I8KWyi5sYVWS4lXiEsA/TGHZg24nplu5zjC11T8SPgyfFupp5WtaK7RG4Cqi3SFl4J46AjA9QcferxOtDQr9NK8Q6ZqMiF0tLuKdlHUhXDEfpX0VqHgHwJ461y88a3niUy2Ny0P7uKdIY0KoqbXZgTzt6fKetcF8VfiLpOo6Ta+E/CStBpNsx+0MkflpKRjaqgdVzkkkcnB9z5FRRRRX//2Q=="" alt="""" />",0
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",4,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkS

Our model ran successfully, and produced a new classification column. These look pretty good - let's filter our Dataframe to show only rows that the model predicted wrongly.

In [11]:
classified_images_df.where(col("label") != col("model_classification")).show(10)

imagePY[list],labelINTEGER,image_2dPY[ndarray],pil_imagePY[Image],model_classificationINTEGER
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",3,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APn+u38DeC9I8Y6dqMMviCLTtaiIa2huMCKVMc5b1zgcdPQ545rXtA1Lwzq82l6tbNb3UXVTyGHZge4PrWbRXoGhfCHX/Eml2moaTf6PcRzqGaNbv95BntIu3g+wzVj4qanYPaeHvD8OqHV9Q0aCSC81DbgOSwwgPfbgjP8AXNeb1u+DdN0vV/F2nWWtX0VlpskmZ5pZBGoUAnG48DOMZ967fQtF8LeBteg1vWPGNndzWkwlt7TQmNxvI5wzkAAdvf1rz/xJqkeueJ9U1WGEwx3l1JOsbHJUMxIB/Osuiiiiv//Z"" alt="""" />",2
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",3,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APHPCfho+K9ZOnLqdhpxETS+deybEOMfKPU85x6A+ldbq/wlhs9Ju7vSvGGj6vc2kDXMtpbPufyl+8w2ls4z3x9a81oorpfBnjfU/A+oXF1p0VrOtzF5U8N1HvR19OCD+v1zWj420zRrnR9L8WaBbNaWupPJFdWWcpbXCYJVD/dIbIHb9BxNFeuaT8JvDvizQrK/8P8AjC3hungU3Fle7S8cvAYcEEDOcfKe3PNUfiPbWXg/wtpXgO2v1vbyC5kv9QkQYVZGXaij0+XPH+NeY0UUUV//2Q=="" alt="""" />",9
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",5,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APn+nwwy3EyQwxvLK5wqIpZmPoAOtdO/w48WRadNfTaT5MUMDXMqTXESSpGDgsYmYOBx6c9q5WivTrm/g+HdpocdhpjSLqVkl1daoshjmuFcZaKGTafKC9CVBJ46VzOveNrrVrKTTrGxttK0yRlaS3tgS05HQyyNlnI9zj2rl6nsrO41G9gsrSIy3M7iOKNerMTgCu/+Hj32uRS+HdXtUufC0G+W6muCU/s7jmSOT+BuPunIY547157OsaXEiwuXiDkI5GNwzwcVHTo5HhlSWJ2SRCGVlOCpHQg1t6x4z8Ra/Zraapqs9xbqwbyzhQzAYBbAG447nJrCor//2Q=="" alt="""" />",8
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",4,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APn+rFhZy6hqNtZQRtJNcSrEiL1YscACtvx7YaTpXjnVtP0NXWwtZvJRXYsQygB+TyfmDVzlFes/DLwVrVjZ3fjWTSLmU2luTpVv5JZrid/lRwvXYu7dnGO/Y1ifEz4fXXgu7sWxe3ST2ySXd48ZMX2li25VfGO2cHmuBre8Gf2Ivi/TX8RzCLSY5fMuCY2cMFBIUhQSQSADx0NehzfGSK5+JC63LFcro2nW8sem2MfAL7SFZ1BA5P1xxjpXA+JPHfiTxZJJ/a2qzywM+8WytthXnIAQccds5PvXOUUUUV//2Q=="" alt="""" />",2
"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",3,"<np.ndarrayshape=(28, 28)dtype=int64>","<img style=""max-height:128px;width:

Some of these look hard indeed, even for a human!

## Analytics

We just managed to run our model, but how well did it actually do? Dataframes expose a powerful set of operations in Groupbys/Aggregations to help us report on aggregates of our data.

Let's group our data by the true labels and calculate how many mistakes our model made per label.

In [12]:
analysis_df = classified_images_df \
    .with_column("correct", (col("model_classification") == col("label")).cast(int)) \
    .with_column("wrong", (col("model_classification") != col("label")).cast(int)) \
    .groupby(col("label")) \
    .agg([
        (col("label").alias("num_rows"), "count"),
        (col("correct"), "sum"),
        (col("wrong"), "sum"),
    ]) \
    .sort(col("label"))

analysis_df.show()

labelINTEGER,num_rowsINTEGER,correctINTEGER,wrongINTEGER
0,980,962,18
1,1135,1121,14
2,1032,992,40
3,1010,967,43
4,982,945,37
5,892,824,68
6,958,934,24
7,1028,965,63
8,974,940,34
9,1009,971,38


Pretty impressive, given that the model only actually trained for one epoch!